# Basic Streams Sample

The basic cells needed are included below.  Make any changes and add your sources, analytics and outputs.

In this notebook, you'll see examples of how to:
 1. [Setup your Streams instance](#setup)
 1. [Create the application](#create)
 1. [Submit the application](#launch)
 1. [Connect to the running application to view data](#view)
 1. [Stop the application](#cancel)

# Overview

**Introduction**

This sample shows how to create and launch an application.

**How it works**
   
The Python application created in this notebook is submitted to the IBM Streams service for execution.

<img src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/04/how-it-works.jpg" alt="How it works">


## Documentation
    
- [Streams Python development guide](https://ibmstreams.github.io/streamsx.documentation/docs/latest/python/)
- [Streams Python API](https://streamsxtopology.readthedocs.io/)



## <a name="setup"> </a> 1. Setup

### 1.1 Add credentials for the IBM Streams service

In order to submit a Streams application you need to provide the name of the Streams instance.

1. From the navigation menu, click **My instances**.
2. Click the **Provisioned Instances** tab.
3. Update the value of `streams_instance_name` in the cell below according to your Streams instance name.


In [2]:
## Paste snippet from development guide
from streamsx.topology.context import ConfigParams
from streamsx.topology import context
import json
import getpass
        
        
service_cfg  = {}
        
SA_credentials=getpass.getpass('Streaming Analytics credentials:')
service_cfg[ConfigParams.SERVICE_DEFINITION] = json.loads(SA_credentials)

def submit_topology(topo):
    global service_cfg
    service_cfg[context.ConfigParams.SSL_VERIFY] = False

    # This specifies how the application will be deployed

    contextType = context.ContextTypes.STREAMING_ANALYTICS_SERVICE

    return context.submit (contextType, topo, config = service_cfg) 

Streaming Analytics credentials:········


### 1.2 Import  the `streamsx` package and verify the package version

In [3]:
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)

#For more details uncomment line below.
#!pip show streamsx

INFO: streamsx package version: 1.13.14


<a id="create"></a>
# 2. Create the application

In [4]:
#create topology...change the name to something more meaningful
from streamsx.topology.topology import Topology
import streamsx.topology.context

topo = Topology(name="BasicTemplate")

2019-12-23 15:32:44,836 - __PROJECT_LIB__ - ERROR - ProjectHandle: Project ID neither provided nor found in the environment.


In [5]:
# add stream...delete and replace this with your streams and analytics
# this is added so the topology will run (needs a source)
# produces a random integer between 1 and 100 every tenth of a second
import random, time
def readings():
    while True:
        time.sleep(0.1)
        yield random.randint(1,100)

source = topo.source(readings)

# create a view to watch the stream data while running
source_view = source.view(name="RandomIntegers", description="Random integers from source stream")

<a id="launch"></a>
# 3. Submit the application
A running Streams application is called a *job*. By submitting the topology we create a job.

In [9]:
#Submit
print("Submitting Topology to Streams for execution..")
submission_result = submit_topology(topo)

if submission_result.job:
    streams_job = submission_result.job
    print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)
else:
    print("Submission failed: "   + str(submssion_result))

Submitting Topology to Streams for execution..


IntProgress(value=0, bar_style='info', description='Initializing', max=10, style=ProgressStyle(description_wid…

JobId:  2 
Job name:  notebook::BasicTemplate_2


<a name="view"></a>
# 4. Use a `View` to access data from the job
Now that the job is started, use the `View` object to start retrieving data from a `Stream`.

In [10]:
#show 30 seconds of data
if submission_result.job:
    queue = source_view.start_data_fetch()
    timeSpan = time.time() + 30;
    while time.time() < timeSpan:
        print(queue.get())
    source_view.stop_data_fetch()


59
92
30
68
85
70
73
26
21
63
54
43
49
53
78
16
46
67
58
66
9
53
18
32
51
94
42
32
31
65
62
10
48
81
56
9
93
1
98
25
89
28
18
68
17
33
95
84
24
71
15
51
7
76
77
11
92
32
56
11
86
12
99
40
86
3
8
72
37
43
74
94
49
29
93
37
100
15
8
100
60
60
92
73
73
70
80
75
24
76
69
11
13
87
17
68
38
79
62
56
6
27
31
25
28
96
1
2
46
33
100
15
55
1
91
47
43
18
25
40
13
48
53
91
46
63
8
83
80
92
80
12
7
38
42
15
64
83
93
36
41
80
65
42
88
68
72
55
90
30
15
73
13
100
60
93
99
98
43
85
60
27
49
71
19
68
42
81
74
73
48
71
40
72
66
21
23
10
79
12
96
17
8
53
1
43
55
22
81
21
93
87
57
94
23
34
44
19
98
45
53
44
2
5
95
54
92
73
36
42
6
79
52
57
22
73
65
34
19
34
24
26
81
1
93
42
43
69
50
34
9
18
29
59
50
11
61
72
56
7
51
69
32
42
28
52
13
94
82
30
63
82
17
48
6
6
33
17
32
92
15
8
81
55
10
76
56
5
98
42
6
79
6
15
29
41
5
2
67
90
91
35
90
37
17
63
37



<a name="cancel"></a>

# 5. Cancel the job

This cell generates widgets you can use to cancel the jobs.

In [11]:
submission_result.cancel_job_button()

You can also interact with the job through the [Job](https://streamsxtopology.readthedocs.io/en/stable/streamsx.rest_primitives.html#streamsx.rest_primitives.Job) object returned from `submission_result.job`

For example, use `streams_job.cancel()` to cancel the running job directly.